In [2]:
import pandas as pd
import numpy as np

# Raw data analysis

In [3]:
data_file = '/Users/Gigio/Desktop/git/explainers/data/paysim_data/PS_20174392719_1491204439457_log.csv'

In [4]:
df=pd.read_csv(data_file)

In [5]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [6]:
df['isFraud'].value_counts()

0    6354407
1       8213
Name: isFraud, dtype: int64

# Data Prep

In [7]:
df = df[['type','amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest','isFraud']]

In [8]:
df=df.join(pd.get_dummies(df['type']))

In [9]:
df.drop('type',axis=1,inplace=True)

In [10]:
df.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,9839.64,170136.0,160296.36,0.0,0.0,0,0,0,0,1,0
1,1864.28,21249.0,19384.72,0.0,0.0,0,0,0,0,1,0
2,181.00,181.0,0.00,0.0,0.0,1,0,0,0,0,1
3,181.00,181.0,0.00,21182.0,0.0,1,0,1,0,0,0
4,11668.14,41554.0,29885.86,0.0,0.0,0,0,0,0,1,0


In [11]:
features=[col for col in df.columns if col!='isFraud']
target='isFraud'

### Split train test

In [12]:
X = df[features].values
y = df[target].values

In [13]:
np.random.shuffle(X)
split=0.7
X_train,y_train=X[:int(split*len(X)),:],y[:int(split*len(X))]
X_test,y_test=X[int(split*len(X)):,:],y[int(split*len(X)):]
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(4453834, 10) (4453834,)
(1908786, 10) (1908786,)


# Train model

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score,precision_score,recall_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

In [15]:
ws_train=compute_sample_weight('balanced',y_train)

In [16]:
parameters = {'n_estimators':(50,100,300), 'max_depth':[3, 10]}
rdf = RandomForestClassifier()
clf = GridSearchCV(rdf, parameters, cv=5)

In [17]:
clf.fit(X_train,y_train)

KeyboardInterrupt: 

In [66]:
y_pred=clf.predict(X_test)

In [67]:
confusion_matrix(y_test,y_pred)

array([[ 495445, 1408771],
       [   1185,    3385]])

In [69]:
accuracy_score(y_test,y_pred)

0.261333643478106